In [1]:
## lstm simple zerostart no feature engineering
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.models import Model
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
from keras.optimizers import RMSprop
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping
%matplotlib inline
import pandas as pd
# importing train and test files
train = pd.read_csv("/Users/isabelmetzger/PycharmProjects/ClinicalScorecard/train_set.txt", 
                  '\t')
train = train.rename(columns = {
#     "sentence": "text",
    "polarity": "label"})
train['id'] = ['TRN' + str(idx) for idx in range(len(train))]
test = pd.read_csv('/Users/isabelmetzger/PycharmProjects/ClinicalScorecard/test_set.txt', '\t')
test = test.rename(columns = {
# "sentence": "text", 
                              "polarity": "label"})
test['id'] = ['TST' + str(idx) for idx in range(len(test))]


train = train.drop_duplicates(subset=None, keep='first', inplace=False)  
train = train.dropna()

def get_binary_target1_stats(train):
    print('number of labeled 1')
    print(train.label.sum())
    print('number of rows')
    print(train.shape[0])
    print('percentage of target')
    print(round(train.label.sum()/train.shape[0]*100))
    
get_binary_target1_stats(train)
train.head()

/Users/isabelmetzger/.local/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


number of labeled 1
651
number of rows
1403
percentage of target
46.0


,sentence,sentiment,label,id
0,"& Kopp, B. T. IFN-gamma stimulates autophagy-m...",8,1,TRN0
1,Pulmonary inflammation induced by Pseudomonas ...,8,1,TRN1
2,These circulating EV-RNA levels have been foun...,8,0,TRN2
3,The company's lead product candidates include ...,8,1,TRN3
4,Its lead product candidate comprises larotrect...,8,0,TRN4


In [2]:
# sentence classifier 
test = test.drop_duplicates(subset=None, keep='first', inplace=False)
test = test.dropna()
get_binary_target1_stats(test)
test.head()

number of labeled 1
258
number of rows
552
percentage of target
47.0


,sentence,sentiment,label,id
0,"In contrast, mice treated with lipid nanoparti...",8,0,TST0
1,The company's lead SARM candidate is the enobo...,8,0,TST1
2,Swappable DNA tiles makes for world's smallest...,8,0,TST2
3,For cells infected with CHIKV E1:226VT percent...,8,0,TST3
4,We have previously observed the up-regulation ...,8,1,TST4


In [4]:
X = train.sentence
y = train.label.values
le = LabelEncoder()
Y = le.fit_transform(y)
Y = y.reshape(-1,1)
print(Y[0:3])

[[1]
 [1]
 [0]]


In [6]:
import keras.backend as K

def binary_accuracy(y_true, y_pred):
    return K.mean(K.equal(y_true, K.round(y_pred)))


def categorical_accuracy(y_true, y_pred):
    return K.mean(K.equal(K.argmax(y_true, axis=-1),
                          K.argmax(y_pred, axis=-1)))


def sparse_categorical_accuracy(y_true, y_pred):
    return K.mean(K.equal(K.max(y_true, axis=-1),
                          K.cast(K.argmax(y_pred, axis=-1), K.floatx())))


def top_k_categorical_accuracy(y_true, y_pred, k=5):
    return K.mean(K.in_top_k(y_pred, K.argmax(y_true, axis=-1), k))


def mean_squared_error(y_true, y_pred):
    return K.mean(K.square(y_pred - y_true))


def mean_absolute_error(y_true, y_pred):
    return K.mean(K.abs(y_pred - y_true))


def mean_absolute_percentage_error(y_true, y_pred):
    diff = K.abs((y_true - y_pred) / K.clip(K.abs(y_true),
                                            K.epsilon(),
                                            None))
    return 100. * K.mean(diff)


def mean_squared_logarithmic_error(y_true, y_pred):
    first_log = K.log(K.clip(y_pred, K.epsilon(), None) + 1.)
    second_log = K.log(K.clip(y_true, K.epsilon(), None) + 1.)
    return K.mean(K.square(first_log - second_log))


def hinge(y_true, y_pred):
    return K.mean(K.maximum(1. - y_true * y_pred, 0.))


def squared_hinge(y_true, y_pred):
    return K.mean(K.square(K.maximum(1. - y_true * y_pred, 0.)))


def categorical_crossentropy(y_true, y_pred):
    return K.mean(K.categorical_crossentropy(y_pred, y_true))


def sparse_categorical_crossentropy(y_true, y_pred):
    return K.mean(K.sparse_categorical_crossentropy(y_pred, y_true))


def binary_crossentropy(y_true, y_pred):
    return K.mean(K.binary_crossentropy(y_pred, y_true))


def kullback_leibler_divergence(y_true, y_pred):
    y_true = K.clip(y_true, K.epsilon(), 1)
    y_pred = K.clip(y_pred, K.epsilon(), 1)
    return K.mean(K.sum(y_true * K.log(y_true / y_pred), axis=-1))


def poisson(y_true, y_pred):
    return K.mean(y_pred - y_true * K.log(y_pred + K.epsilon()))


def cosine_proximity(y_true, y_pred):
    y_true = K.l2_normalize(y_true, axis=-1)
    y_pred = K.l2_normalize(y_pred, axis=-1)
    return -K.mean(y_true * y_pred)

def matthews_correlation(y_true, y_pred):
    """Matthews correlation metric.
    It is only computed as a batch-wise average, not globally.
    Computes the Matthews correlation coefficient measure for quality
    of binary classification problems.
    """
    y_pred_pos = K.round(K.clip(y_pred, 0, 1))
    y_pred_neg = 1 - y_pred_pos

    y_pos = K.round(K.clip(y_true, 0, 1))
    y_neg = 1 - y_pos

    tp = K.sum(y_pos * y_pred_pos)
    tn = K.sum(y_neg * y_pred_neg)

    fp = K.sum(y_neg * y_pred_pos)
    fn = K.sum(y_pos * y_pred_neg)

    numerator = (tp * tn - fp * fn)
    denominator = K.sqrt((tp + fp) * (tp + fn) * (tn + fp) * (tn + fn))

    return numerator / (denominator + K.epsilon())


def precision(y_true, y_pred):
    """Precision metric.
    Only computes a batch-wise average of precision.
    Computes the precision, a metric for multi-label classification of
    how many selected items are relevant.
    """
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision


def recall(y_true, y_pred):
    """Recall metric.
    Only computes a batch-wise average of recall.
    Computes the recall, a metric for multi-label classification of
    how many relevant items are selected.
    """
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall


def fbeta_score(y_true, y_pred, beta=1):
    """Computes the F score.
    The F score is the weighted harmonic mean of precision and recall.
    Here it is only computed as a batch-wise average, not globally.
    This is useful for multi-label classification, where input samples can be
    classified as sets of labels. By only using accuracy (precision) a model
    would achieve a perfect score by simply assigning every class to every
    input. In order to avoid this, a metric should penalize incorrect class
    assignments as well (recall). The F-beta score (ranged from 0.0 to 1.0)
    computes this, as a weighted mean of the proportion of correct class
    assignments vs. the proportion of incorrect class assignments.
    With beta = 1, this is equivalent to a F-measure. With beta < 1, assigning
    correct classes becomes more important, and with beta > 1 the metric is
    instead weighted towards penalizing incorrect class assignments.
    """
    if beta < 0:
        raise ValueError('The lowest choosable beta is zero (only precision).')

    # If there are no true positives, fix the F score at 0 like sklearn.
    if K.sum(K.round(K.clip(y_true, 0, 1))) == 0:
        return 0

    p = precision(y_true, y_pred)
    r = recall(y_true, y_pred)
    bb = beta ** 2
    fbeta_score = (1 + bb) * (p * r) / (bb * p + r + K.epsilon())
    return fbeta_score


def fmeasure(y_true, y_pred):
    """Computes the f-measure, the harmonic mean of precision and recall.
    Here it is only computed as a batch-wise average, not globally.
    """
    return fbeta_score(y_true, y_pred, beta=1)


# aliases
mse = MSE = mean_squared_error
mae = MAE = mean_absolute_error
mape = MAPE = mean_absolute_percentage_error
msle = MSLE = mean_squared_logarithmic_error
cosine = cosine_proximity
fscore = f1score = fmeasure
print("K backend imported!")

# le = LabelEncoder()
# Y = le.fit_transform(y)
# Y_train = y.reshape(-1,1)

t = train['sentence'][3]  
t

K backend imported!


"The company's lead product candidates include PTG-100, an oral alpha-4-beta-7 integrin- antagonist that is in Phase II b clinical trial for the treatment of ulcerative colitis, as well as for treating chronic pouchitis, a gastrointestinal (GI) condition that occurs in post-surgical inflammatory bowel disease (IBD) patients; PTG-200, an oral interleukin-23 receptor antagonist, which is in Phase I clinical trial for the treatment of IBD; and PTG-300, an injectable hepcidin mimetic, which has completed Phase I study for use in the treatment of beta-thalassemia, as well as for treating other diseases, such as hereditary hemochromatosis, polycythemia vera, siderophilic infections, and liver fibrosis."

In [7]:
import spacy
from scispacy.abbreviation import AbbreviationDetector
from scispacy.umls_linking import UmlsEntityLinker
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

nlp = spacy.load('en_core_sci_md')
abbreviation_pipe = AbbreviationDetector(nlp)
nlp.add_pipe(abbreviation_pipe)
linker = UmlsEntityLinker()
nlp.add_pipe(linker)
  
#     """
#     return doc.to_json()

def get_list_ents(doc):
    return [[ent.text, ent.start, ent.end, ent.label_] for ent in doc.ents]


doc = nlp(t)
get_list_ents(doc)

/Users/isabelmetzger/anaconda3/lib/python3.6/site-packages/sklearn/base.py:306: UserWarning: Trying to unpickle estimator TfidfTransformer from version 0.20.3 when using version 0.21.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/Users/isabelmetzger/anaconda3/lib/python3.6/site-packages/sklearn/base.py:306: UserWarning: Trying to unpickle estimator TfidfVectorizer from version 0.20.3 when using version 0.21.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


[["company's", 1, 3, 'ENTITY'],
 ['PTG-100', 7, 8, 'ENTITY'],
 ['oral', 10, 11, 'ENTITY'],
 ['antagonist', 13, 14, 'ENTITY'],
 ['Phase II b', 17, 20, 'ENTITY'],
 ['clinical trial', 20, 22, 'ENTITY'],
 ['treatment', 24, 25, 'ENTITY'],
 ['ulcerative colitis', 26, 28, 'ENTITY'],
 ['treating', 33, 34, 'ENTITY'],
 ['chronic pouchitis', 34, 36, 'ENTITY'],
 ['gastrointestinal (', 38, 40, 'ENTITY'],
 ['GI', 40, 41, 'ENTITY'],
 ['condition', 42, 43, 'ENTITY'],
 ['post-surgical', 46, 47, 'ENTITY'],
 ['inflammatory bowel disease', 47, 50, 'ENTITY'],
 ['IBD', 51, 52, 'ENTITY'],
 ['patients', 53, 54, 'ENTITY'],
 ['PTG-200', 55, 56, 'ENTITY'],
 ['oral interleukin-23 receptor antagonist', 58, 62, 'ENTITY'],
 ['Phase I', 66, 68, 'ENTITY'],
 ['treatment', 72, 73, 'ENTITY'],
 ['IBD', 74, 75, 'ENTITY'],
 ['PTG-300', 77, 78, 'ENTITY'],
 ['injectable', 80, 81, 'ENTITY'],
 ['hepcidin', 81, 82, 'ENTITY'],
 ['mimetic', 82, 83, 'ENTITY'],
 ['Phase I study', 87, 90, 'ENTITY'],
 ['treatment', 94, 95, 'ENTITY'],


In [8]:
import en_ner_jnlpba_md
import en_ner_craft_md
import en_ner_bionlp13cg_md
import en_ner_bc5cdr_md
import en_core_web_lg

jnlpba = en_ner_jnlpba_md.load()

get_list_ents(jnlpba(t))

[['PTG-100', 7, 8, 'DNA'], ['oral interleukin-23 receptor', 58, 61, 'PROTEIN']]

In [9]:
craft = en_ner_craft_md.load()
get_list_ents(craft(t))

[['antagonist', 13, 14, 'CHEBI'],
 ['interleukin-23 receptor', 59, 61, 'GGP'],
 ['antagonist', 61, 62, 'CHEBI'],
 ['PTG-300', 77, 78, 'CL'],
 ['hepcidin', 81, 82, 'GGP']]

In [10]:
bc5cdr = en_ner_bc5cdr_md.load()
get_list_ents(bc5cdr(t))

[['PTG-100', 7, 8, 'CHEMICAL'],
 ['ulcerative colitis', 26, 28, 'DISEASE'],
 ['pouchitis', 35, 36, 'DISEASE'],
 ['gastrointestinal (GI) condition that occurs in post-surgical inflammatory bowel disease',
  38,
  50,
  'DISEASE'],
 ['IBD', 51, 52, 'DISEASE'],
 ['IBD', 74, 75, 'DISEASE'],
 ['beta-thalassemia', 96, 97, 'DISEASE'],
 ['hereditary hemochromatosis', 108, 110, 'DISEASE'],
 ['polycythemia vera', 111, 113, 'DISEASE'],
 ['siderophilic infections', 114, 116, 'DISEASE'],
 ['liver fibrosis', 118, 120, 'DISEASE']]

In [12]:
linguistic_features_for_embeddings = [[token.text, token.lemma_, token.pos_, token.tag_, token.dep_,
            token.shape_, token.is_alpha, token.is_stop] for token in doc]
pd.DataFrame(linguistic_features_for_embeddings).head(15).rename(columns={0:'text', 1: 'lemma_', 2: 'pos_', 3: 'tag_', 4: 
                                                                         'dep_', 5: 'shape_', 6: 'is_alpha', 7: 'is_stop'})

,text,lemma_,pos_,tag_,dep_,shape_,is_alpha,is_stop
0,The,the,DET,DT,det,Xxx,True,True
1,company,company,NOUN,NN,poss,xxxx,True,False
2,'s,'s,PART,POS,case,'x,False,True
3,lead,lead,NOUN,NN,compound,xxxx,True,False
4,product,product,NOUN,NN,compound,xxxx,True,False
5,candidates,candidate,NOUN,NNS,nsubj,xxxx,True,False
6,include,include,VERB,VBP,ROOT,xxxx,True,False
7,PTG-100,ptg-100,NOUN,NN,dobj,XXX-ddd,False,False
8,",",",",PUNCT,",",punct,",",False,False
9,an,an,DET,DT,det,xx,True,True


In [17]:
print("===================")
print(doc.ents[4])
print("===================")
entity_phase_II_b = doc.ents[4]
for umls_ent in entity_phase_II_b._.umls_ents:

    print(linker.umls.cui_to_entity[umls_ent[0]])
print("===================")
print("===================")
print(doc.ents[28])
print("===================")
beta_thalassemia = doc.ents[28]
for umls_ent in beta_thalassemia._.umls_ents:
    print(linker.umls.cui_to_entity[umls_ent[0]])

Phase II b
CUI: C1706096, Name: Phase II B Trial
Definition: A clinical research protocol generally referred to as a well-controlled and pivotal trial that aims to prove the mechanism of action of the new intervention in question. A pivotal study will generally be well-controlled, randomized, of adequate size, and whenever possible, double-blind.
TUI(s): T062
Aliases: (total: 6): 
	 2B, Trial Phase 2B, Trial Phase 2B, Phase IIb Trial, Trial Phase IIB, PHASE IIB TRIAL
CUI: C0282460, Name: Phase 2 Clinical Trials
Definition: Works about studies that are usually controlled to assess the effectiveness and dosage (if appropriate) of diagnostic, therapeutic, or prophylactic drugs, devices, or techniques. These studies are performed on several hundred volunteers, including a limited number of patients with the target disease or disorder, and last about two years. This concept includes phase II studies conducted in both the U.S. and in other countries.
TUI(s): T062
Aliases (abbreviated, total:

In [18]:
[[abbrev, abbrev.start, abbrev.end, abbrev._.long_form] for abbrev in doc._.abbreviations]

[[GI, 40, 41, gastrointestinal],
 [IBD, 74, 75, inflammatory bowel disease],
 [IBD, 51, 52, inflammatory bowel disease]]

In [19]:
[[ent.text, ent._.umls_ents] for ent in doc.ents]

[["company's", []],
 ['PTG-100',
  [('C3812281', 0.9009826183319092), ('C1414627', 0.9009826183319092)]],
 ['oral',
  [('C0442027', 1.0),
   ('C1527415', 1.0),
   ('C1272919', 1.0),
   ('C0226896', 1.0)]],
 ['antagonist',
  [('C0231491', 1.0),
   ('C0016411', 0.8194425106048584),
   ('C0036753', 0.8177525401115417),
   ('C0019593', 0.803914487361908),
   ('C0597668', 0.8030994534492493)]],
 ['Phase II b',
  [('C1706096', 0.8509665131568909),
   ('C0282460', 0.7450801730155945),
   ('C0205390', 0.7105843424797058)]],
 ['clinical trial',
  [('C1096775', 1.0),
   ('C0008976', 1.0),
   ('C1552009', 1.0),
   ('C2983596', 0.8558200597763062),
   ('C2827983', 0.8512393236160278)]],
 ['treatment',
  [('C3538994', 1.0),
   ('C3161471', 1.0),
   ('C0039798', 1.0),
   ('C3887704', 1.0),
   ('C0087111', 1.0)]],
 ['ulcerative colitis',
  [('C0009324', 0.9999999403953552),
   ('C4053902', 0.8694170117378235),
   ('C0041582', 0.8100262880325317)]],
 ['treating', [('C1522326', 1.0), ('C2985688', 0.769

In [20]:
import re

sci_md = spacy.load("en_core_sci_md")
def clean_text(text):
    doc = sci_md(text)
    text_out = " ".join([token.text.lower() for token in doc])
    return text_out

train['text'] = train['sentence'].apply(lambda x: clean_text(str(x)))
test['text'] = test['sentence'].apply(lambda x: clean_text(str(x)))

In [21]:
X = train.text
test_X = test.text
print(X.shape, test_X.shape) 
X_train = train.text
y_train = train.label.values
X_test = test.text
y_test = test.label.values
le = LabelEncoder()
Y_train = le.fit_transform(y_train)
print(X_train.sample(5))
le = LabelEncoder()
Y_test = le.fit_transform(y_test)
train['Ntokens'] = train['text'].apply(lambda x: len(x.split(' ')))
test['Ntokens'] = test['text'].apply(lambda x: len(x.split(' ')))

np.max(train['Ntokens'])

(1403,) (552,)
31     by a pilot study with small population , il-17...
27     results : the spect/ct and mri were found to h...
446    ( c ) aberrant cdx2 expression in squamoid mor...
700    for needs assessment of these specific tumors ...
204    the companyâ ’s product candidates include x35...
Name: text, dtype: object


257

In [34]:
max_words = 3500
max_len = 3500
tok = Tokenizer(num_words=max_words)
tok.fit_on_texts(X_train)
sequences = tok.texts_to_sequences(X_train)
sequences_matrix = sequence.pad_sequences(sequences,maxlen=max_len)
print("Sequences Matrix Made!")

Sequences Matrix Made!


In [35]:
def RNN():
    inputs = Input(name='inputs',shape=[max_len])
    layer = Embedding(max_words,100,input_length=max_len)(inputs)
    layer = LSTM(64)(layer)
    layer = Dense(256,name='FC1')(layer)
    layer = Activation('relu')(layer)
    layer = Dropout(0.2)(layer)
    layer = Dense(1,name='out_layer')(layer)
    layer = Activation('sigmoid')(layer)
    model = Model(inputs=inputs,outputs=layer)
    return model
model = RNN()
model.summary()
model.compile(loss='binary_crossentropy',optimizer=RMSprop(),metrics=['accuracy', precision, recall, fmeasure])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inputs (InputLayer)          (None, 3500)              0         
_________________________________________________________________
embedding_4 (Embedding)      (None, 3500, 100)         350000    
_________________________________________________________________
lstm_4 (LSTM)                (None, 64)                42240     
_________________________________________________________________
FC1 (Dense)                  (None, 256)               16640     
_________________________________________________________________
activation_7 (Activation)    (None, 256)               0         
_________________________________________________________________
dropout_4 (Dropout)          (None, 256)               0         
_________________________________________________________________
out_layer (Dense)            (None, 1)                 257       
__________

In [36]:
model.fit(sequences_matrix,Y_train,batch_size=32,epochs=24,
          validation_split=0.2,callbacks=[EarlyStopping(monitor='val_loss',min_delta=0.0001)])

Train on 1122 samples, validate on 281 samples
Epoch 1/24
1122/1122 [==============================] - 97s 86ms/step - loss: 0.6779 - acc: 0.6230 - precision: 0.6025 - recall: 0.4948 - fmeasure: 0.4760 - val_loss: 0.6479 - val_acc: 0.5836 - val_precision: 0.4555 - val_recall: 0.0790 - val_fmeasure: 0.1329
Epoch 2/24
1122/1122 [==============================] - 100s 89ms/step - loss: 0.3779 - acc: 0.8431 - precision: 0.8493 - recall: 0.8330 - fmeasure: 0.8247 - val_loss: 0.5598 - val_acc: 0.7367 - val_precision: 0.7011 - val_recall: 0.6655 - val_fmeasure: 0.6796
Epoch 3/24
1122/1122 [==============================] - 95s 85ms/step - loss: 0.1978 - acc: 0.9269 - precision: 0.9401 - recall: 0.9073 - fmeasure: 0.9155 - val_loss: 0.6317 - val_acc: 0.7189 - val_precision: 0.6946 - val_recall: 0.6189 - val_fmeasure: 0.6483


In [37]:
test_sequences = tok.texts_to_sequences(X_test)
test_sequences_matrix = sequence.pad_sequences(test_sequences,maxlen=max_len)
accr = model.evaluate(test_sequences_matrix,Y_test)

print('Val set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1], accr[2], accr[3]))

552/552 [==============================] - 7s 13ms/step
Val set
  Loss: 0.571
  Accuracy: 0.772


In [38]:
## SECOND MODEL

In [ ]:
import nltk
nltk.download('vader_lexicon')
print('loading embeddings vectors')
def get_coefs(word,*arr): 
    return word, np.asarray(arr, dtype='float32')

embeddings_index = dict(get_coefs(*o.split(' '))
        for o in open('/embeddings/glove.840B.300d.txt'))



batch_size = 512 
epochs = 4
embed_size = 300 #embeddings dimension
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sia = SentimentIntensityAnalyzer()
print("done!")
# importing libraries
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
%matplotlib inline
import seaborn as sns
import re
# Fork of Sergei Fironov's script CNN GLOVE300 3-OOF 3 epochs
import os
os.environ['OMP_NUM_THREADS'] = '4'
import tensorflow as tf
from keras.models import Model
from keras.layers import Dense, Embedding, Input, Concatenate, Conv1D, Activation, TimeDistributed, Flatten, RepeatVector, Permute,multiply
from keras.layers import LSTM, Bidirectional, GlobalMaxPool1D, Dropout, GRU, GlobalAveragePooling1D, MaxPooling1D, SpatialDropout1D, BatchNormalization
from keras.preprocessing import text, sequence
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.optimizers import Adam
from keras.preprocessing.text import Tokenizer
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import KFold
from sklearn import metrics
list_sentences_train = train["text"].fillna("").values #(30053,)
print(list_sentences_train.shape) # print(list_sentences_train.shape) # 
y = train["label"].values
max_features = 150#15500 #
maxlen = 7000 #padding length
num_folds = 3#2 #number of folds
list_sentences_test = test["text"].fillna("").values
from keras.preprocessing.text import Tokenizer
print('mean text len:',train["text"].str.count('\S+').mean())
print('max text len:',train["text"].str.count('\S+').max())
min_count =2
tokenizer = Tokenizer()
tokenizer.fit_on_texts(list(list_sentences_train) + list(list_sentences_test))
num_words = sum([1 for _, v in tokenizer.word_counts.items() if v >= min_count])
print('num_words',num_words)
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list(list_sentences_train)) # + list(list_sentences_test)
list_tokenized_train = tokenizer.texts_to_sequences(list_sentences_train)
list_tokenized_test = tokenizer.texts_to_sequences(list_sentences_test)
print('padding sequences')
X_train = {}
X_test = {}
X_train['sentence'] = sequence.pad_sequences(list_tokenized_train, maxlen=maxlen, padding='post', truncating='post')
X_test['sentence'] = sequence.pad_sequences(list_tokenized_test, maxlen=maxlen, padding='post', truncating='post')

print('numerical variables')
train['num_words'] = train.sentence.str.count('\S+')
test['num_words'] = test.sentence.str.count('\S+')

rain['avg_word'] = train.sentence.str.len() / (1 + train.num_words)
test['avg_word'] = test.sentence.str.len() / (1 + test.num_words)
print('sentiment')

train['num_comas'] = train.sentence.str.count('\.')
test['num_comas'] = test.sentence.str.count('\.')
train['num_bangs'] = train.sentence.str.count('\!')
test['num_bangs'] = test.sentence.str.count('\!')
train['num_quotas'] = train.sentence.str.count('\"')
test['num_quotas'] = test.sentence.str.count('\"')
scaler = MinMaxScaler()

X_train['num_vars'] = scaler.fit_transform(train[['SentimentScores','num_words','num_comas','num_bangs','num_quotas','avg_word','len', 'qmark', 'SentimentScores', 'Ncommas',
       'Nsemicolumns', 'Ncolons', 'Nblank', 'Nother', 'Ncapitalfirst', 
       'Ncapital', 'Nnumber', 'Nwords', 'Nunique', 'Nrepeated',
       'UniquenessRatio', 'Wmean', 'Wmedian', 'Wsd']])

X_test['num_vars'] = scaler.transform(test[['SentimentScores','num_words','num_comas','num_bangs','num_quotas','avg_word','len', 'qmark', 'SentimentScores', 'Ncommas',
       'Nsemicolumns', 'Ncolons', 'Nblank', 'Nother', 'Ncapitalfirst',
       'Ncapital', 'Nnumber', 'Nwords', 'Nunique', 'Nrepeated',
       'UniquenessRatio', 'Wmean', 'Wmedian', 'Wsd']])

all_embs = np.stack(embeddings_index.values())
emb_mean,emb_std = all_embs.mean(), all_embs.std()

print('create embedding matrix')
word_index = tokenizer.word_index
nb_words = min(max_features, len(word_index))
embedding_matrix = np.random.normal(emb_mean, emb_std, (nb_words, embed_size))
for word, i in word_index.items():
    if i >= max_features: continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None: embedding_matrix[i] = embedding_vector

def get_model_cnn(X_train):
    global embed_size
    inp = Input(shape=(maxlen, ), name="text")
    num_vars = Input(shape=[X_train["num_vars"].shape[1]], name="num_vars")
    x = Embedding(max_features, embed_size, weights=[embedding_matrix])(inp)
    x = SpatialDropout1D(0.2)(x)
    z = GlobalMaxPool1D()(x)
    x = GlobalMaxPool1D()(Conv1D(embed_size, 4, activation="relu")(x))
    x = Concatenate()([x,z,num_vars])
    x = Dropout(0.3)(x)
    x = Dense(1, activation="sigmoid")(x)
    model = Model(inputs=[inp,num_vars], outputs=x)
    model.compile(loss=binary_crossentropy, optimizer='adam', metrics=['accuracy', precision, recall, fmeasure])
    return model   

print("GLOVE KERAS SIMPLE MODEL! WITH FE! 300Dimension Glovee\n")
print('start modeling')
scores = []
predict = np.zeros((test.shape[0],1))
oof_predict = np.zeros((train.shape[0],1))
f1_scores = []
acc_scores = []
kf = KFold(n_splits=num_folds, shuffle=True, random_state=239)
for train_index, test_index in kf.split(X_train['num_vars']):
    kfold_X_train = {}
    kfold_X_valid = {}
    y_train,y_test = y[train_index], y[test_index]
    for c in ['text','num_vars']:
        kfold_X_train[c] = X_train[c][train_index]
        kfold_X_valid[c] = X_train[c][test_index]

    model = get_model_cnn(X_train)
    model.fit(kfold_X_train, y_train, batch_size=batch_size, epochs=epochs, verbose=1)
    predict += model.predict(X_test, batch_size=1000) / num_folds
    oof_predict[test_index] = model.predict(kfold_X_valid, batch_size=1000)
    
    cv_score = metrics.roc_auc_score(y_test, oof_predict[test_index])
    fscore =  f1_score(y_test, oof_predict[test_index])
    metrics.average_precision_score(y_test, oof_predict[test_index])
    print(model.metrics_names)#, model.metrics_tensors)
    scores.append(cv_score)
    f1_scores.append(fscore)
    print('cv score: ', cv_score)
    print('f1 score: ',fscore)
    print(classification_report(y_test, oof_predict[test_index]))

print('Total CV score is {}'.format(np.mean(scores)))    
print('Mean f1 score is {}'.format(np.mean(f1_scores))) 
